In [2]:
import sys
import os
from pathlib import Path
import tensorflow as tf
import numpy as np

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.architecture as ARCH
import SnowDepth.optimal_features as OF

In [ ]:
# Assign seed
seed = 18

# Select holdout AOI
holdout_aoi = "ID_BS"

# H5 directory and expected output files
h5_dir = ROOT / "data" / "h5_dir"
h5_path_HSIC = h5_dir / "HSIC" / "data_HSIC.h5"
h5_path_PCC  = h5_dir / "PCC"  / "data_PCC.h5"
h5_path_MI   = h5_dir / "MI"   / "data_MI.h5"

# If we have already written H5 files, we can skip a lot of steps here:
if all(p.exists() for p in [h5_path_HSIC, h5_path_PCC, h5_path_MI]):
    print("All H5 files already exist. Skipping feature selection and H5 generation.")
else:
    print("H5 files missing. Running feature selection and writing new H5 files")

    # SET IMPORTANT VARIABLES HERE:
    
    # Path to TIFF files
    data_dir = ROOT / "data" / "tif_files"

    # Number of features to select
    top_k = 10

    # Load dataframe
    df = DL.build_df(str(data_dir), drop_invalid=True, upper_threshold=3)
    dev_df = df[df["aoi_name"] != holdout_aoi].copy()
    hold_df = df[df["aoi_name"] == holdout_aoi].copy()

    # Run feature selection
    ff_algos = OF.optimal_feature_sets(dev_df, top_k=top_k, n_per_aoi=10000)

    # Write one H5 file per feature-set
    for name, feats in ff_algos.items():
        out_dir = h5_dir / name
        out_dir.mkdir(parents=True, exist_ok=True)
        DL.build_h5(
            data_dir=str(data_dir),
            out_dir=str(out_dir),
            write_mask=True,
            upper_threshold=3.0,
            selected_features=feats,
            out_name=f"data_{name}.h5",
        )
        print(f"Wrote new H5: {out_dir / f'data_{name}.h5'}")

In [ ]:
# HSIC UNET data split
(X_train_HSIC, y_train_HSIC, m_train_HSIC), (X_val_HSIC, y_val_HSIC, m_val_HSIC) = DS.DL_split(
    h5_path=str(h5_path_HSIC),
    holdout_aoi=holdout_aoi,
    val_fraction=0.10,       
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("HSIC Shapes:",
      "X_train", X_train_HSIC.shape, "y_train", y_train_HSIC.shape,
      "X_val",   X_val_HSIC.shape,   "y_val",   y_val_HSIC.shape)

(X_train_n_HSIC, X_val_n_HSIC) = ARCH.zscore_from_train(X_train_HSIC, X_val_HSIC)
y_train_f_HSIC, w_train_HSIC = ARCH.fill_nan_and_mask(y_train_HSIC)
y_val_f_HSIC,   w_val_HSIC   = ARCH.fill_nan_and_mask(y_val_HSIC)
X_train_n_HSIC = X_train_n_HSIC.astype("float32"); X_val_n_HSIC = X_val_n_HSIC.astype("float32")

# HSIC UNet training
model_HSIC = ARCH.unet(input_shape=X_train_n_HSIC.shape[1:], base_filters=32)
LR = 1e-3
model_HSIC.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)
os.makedirs("UNet_weights", exist_ok=True)
ckpt_HSIC = "UNet_weights/unet_best_HSIC.weights.h5"

w_train_HSIC_4d = w_train_HSIC[..., None].astype("float32")
w_val_HSIC_4d   = w_val_HSIC[..., None].astype("float32")

callbacks_HSIC = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_HSIC, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]
hist_HSIC = model_HSIC.fit(
    X_train_n_HSIC, y_train_f_HSIC,
    sample_weight=w_train_HSIC_4d,
    validation_data=(X_val_n_HSIC, y_val_f_HSIC, w_val_HSIC_4d),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_HSIC,
    verbose=1,
)

HSIC Shapes: X_train (289, 128, 128, 10) y_train (289, 128, 128, 1) X_val (32, 128, 128, 10) y_val (32, 128, 128, 1)
Epoch 1/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - MAE: 0.9186 - loss: 0.5408
Epoch 1: val_loss improved from inf to 5.10073, saving model to UNet_weights/unet_best_HSIC.weights.h5
73/73 ━━━━━━━━━━━━━━━━━━━━ 40s 377ms/step - MAE: 0.9148 - loss: 0.5377 - val_MAE: 5.5960 - val_loss: 5.1007 - learning_rate: 0.0010
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - MAE: 0.4538 - loss: 0.1477
Epoch 2: val_loss improved from 5.10073 to 0.93791, saving model to UNet_weights/unet_best_HSIC.weights.h5
73/73 ━━━━━━━━━━━━━━━━━━━━ 26s 352ms/step - MAE: 0.4535 - loss: 0.1476 - val_MAE: 1.3162 - val_loss: 0.9379 - learning_rate: 0.0010
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - MAE: 0.4019 - loss: 0.1198
Epoch 3: val_loss improved from 0.93791 to 0.15481, saving model to UNet_weights/unet_best_HSIC.weights.h5
73/73 ━━━━━━━━━━━━━━━━━━━━ 24s 327ms/step - MAE: 0.4020 - l

In [ ]:
# PCC UNET data split
(X_train_PCC, y_train_PCC, m_train_PCC), (X_val_PCC, y_val_PCC, m_val_PCC) = DS.unet_split(
    h5_path=str(h5_path_PCC),
    holdout_aoi=holdout_aoi,
    val_fraction=0.10,
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("PCC  Shapes:",
      "X_train", X_train_PCC.shape, "y_train", y_train_PCC.shape,
      "X_val",   X_val_PCC.shape,   "y_val",   y_val_PCC.shape)

(X_train_n_PCC, X_val_n_PCC) = ARCH.zscore_from_train(X_train_PCC, X_val_PCC)
y_train_f_PCC, w_train_PCC = ARCH.fill_nan_and_mask(y_train_PCC)
y_val_f_PCC,   w_val_PCC   = ARCH.fill_nan_and_mask(y_val_PCC)
X_train_n_PCC = X_train_n_PCC.astype("float32"); X_val_n_PCC = X_val_n_PCC.astype("float32")

# PCC UNet training
model_PCC = ARCH.unet(input_shape=X_train_n_PCC.shape[1:], base_filters=32)
model_PCC.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)
ckpt_PCC = "UNet_weights/unet_best_PCC.weights.h5"

w_train_PCC_4d = w_train_PCC[..., None].astype("float32")
w_val_PCC_4d   = w_val_PCC[..., None].astype("float32")

callbacks_PCC = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_PCC, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]
hist_PCC = model_PCC.fit(
    X_train_n_PCC, y_train_f_PCC,
    sample_weight=w_train_PCC_4d,
    validation_data=(X_val_n_PCC, y_val_f_PCC, w_val_PCC_4d),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_PCC,
    verbose=1,
)


In [ ]:
# MI UNET data split
(X_train_MI, y_train_MI, m_train_MI), (X_val_MI, y_val_MI, m_val_MI) = DS.unet_split(
    h5_path=str(h5_path_MI),
    holdout_aoi=holdout_aoi,
    val_fraction=0.10,
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("MI   Shapes:",
      "X_train", X_train_MI.shape, "y_train", y_train_MI.shape,
      "X_val",   X_val_MI.shape,   "y_val",   y_val_MI.shape)

(X_train_n_MI, X_val_n_MI) = ARCH.zscore_from_train(X_train_MI, X_val_MI)
y_train_f_MI, w_train_MI = ARCH.fill_nan_and_mask(y_train_MI)
y_val_f_MI,   w_val_MI   = ARCH.fill_nan_and_mask(y_val_MI)
X_train_n_MI = X_train_n_MI.astype("float32"); X_val_n_MI = X_val_n_MI.astype("float32")



# MI UNet training
Unet_MI = ARCH.unet(input_shape=X_train_n_MI.shape[1:], base_filters=32)
Unet_MI.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)
ckpt_Unet_MI = "UNet_weights/unet_best_MI.weights.h5"

w_train_MI_4d = w_train_MI[..., None].astype("float32")
w_val_MI_4d   = w_val_MI[..., None].astype("float32")

callbacks_MI = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_Unet_MI, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]
hist_MI = Unet_MI.fit(
    X_train_n_MI, y_train_f_MI,
    sample_weight=w_train_MI_4d,
    validation_data=(X_val_n_MI, y_val_f_MI, w_val_MI_4d),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_MI,
    verbose=1,
)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - MAE: 0.9361 - loss: 0.5546
Epoch 1: val_loss improved from inf to 156.48790, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 364ms/step - MAE: 0.9309 - loss: 0.5502 - val_MAE: 160.2921 - val_loss: 156.4879 - learning_rate: 0.0010
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - MAE: 0.4539 - loss: 0.1459
Epoch 2: val_loss improved from 156.48790 to 4.71764, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 19s 350ms/step - MAE: 0.4536 - loss: 0.1457 - val_MAE: 5.2918 - val_loss: 4.7176 - learning_rate: 0.0010
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - MAE: 0.4295 - loss: 0.1348
Epoch 3: val_loss improved from 4.71764 to 0.53150, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 18s 339ms/step - MAE: 0.4294 - loss: 0.1348 - val_MAE: 0.9598 - val_loss: 0.5315 - learning_rate: 0.0010
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - MAE: 0.4201 